# Build a Dashboard Application with Plotly Dash


In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component\ TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown\ TASK 3: Add a Range Slider to Select Payload\ TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

In [1]:
!pip install pandas dash


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


--2022-06-12 23:04:21--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-06-12 23:04:21 (36.7 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [5]:
# Create a dash application
app = dash.Dash(__name__)

In [7]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                  dcc.Dropdown(id='site-dropdown',
                                             options=[{'label':'All Sites', 'value': 'ALL'},
                                                      {'label':'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label':'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                      {'label':'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label':'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                          ],
                                            value='ALL',
                                            placeholder="Select a Launch Site",
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'},
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload Range (kg):"),

                                # TASK 3: Add a slider to select payload range
                                 dcc.RangeSlider(min=0,max=10000, step = 1000,
                                marks={0: {'label': '0 kg', 'style': {'color': 'Red'}},
                                       2500: {'label': '2500 kg'},
                                       5000: {'label': '5000 kg'},
                                       7500: {'label': '7500 kg'},
                                       10000: {'label': '10000 kg'}, 'style': {'color': 'Dark Blue'}
                                }, value=[min_payload, max_payload],
                                id='payload-slider'),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value')
)
def get_pie(value):
    if value == 'ALL':
        fig1 = px.pie(spacex_df, values='class', names='Launch Site',\
             title='Total Success Launches By Each Site')
        return fig1

    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == value].groupby(['Launch Site', 'class']).\
        size().reset_index(name='class count')
        title = f"Total Success Launches for site {value}"
        fig2 = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig2

    # TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'),
Input(component_id='payload-slider', component_property='value')]
)

def get_scatter(value1, value2):
    scatas_df=spacex_df[(spacex_df['Payload Mass (kg)'] >= value2[0]) &\
         (spacex_df['Payload Mass (kg)'] <= value2[1])]

    if value1=='ALL':
        fig3= px.scatter(scatas_df,x="Payload Mass (kg)", y="class",\
            color="Booster Version Category",\
        title="Correlation between Payload and Success for All Sites")
        return fig3
    else:
        scatss_df=scatas_df[scatas_df['Launch Site']==value1]
        fig4= px.scatter(scatss_df, x="Payload Mass (kg)", y="class",\
            color="Booster Version Category",\
        title=f"Correlation between Payload and Success for site {value1}")
        return fig4

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://x86_64-conda_cos6-linux-gnu:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Name or service not known
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/werkzeug/serving.py", line 908, in prepare_socket
    s.bind(server_address)
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_381/2615491575.py", line 88, in <module>
    app.run_server()
  File "/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/dash/dash.py", line 2065, in run_server
    self.run(*args, **kwargs)
  File "/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/dash/dash.py", line 2057, in run
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/jupyterlab/conda/envs/python/lib/python3.7/site-package

TypeError: object of type 'NoneType' has no len()